In [2]:
from langchain_community.document_loaders import PyPDFLoader
import json
import os

In [3]:
def carregar_documentos(caminho_arquivo):
    loader = PyPDFLoader(caminho_arquivo)
    documentos = loader.load_and_split()
    return documentos

def limpar_texto(texto):
    return texto.strip()

def extrair_informacoes(texto):
    linhas = texto.split("\n")
    informacoes = {}
    
    chave_atual = None
    lista_valores = []

    for linha in linhas:
        linha = linha.strip()
        if ":" in linha and not linha.startswith("-"):
            if chave_atual:
                informacoes[chave_atual] = lista_valores if lista_valores else valor.strip()
            chave, valor = linha.split(":", 1)
            chave = chave.strip().replace(" ", "-")
            valor = valor.strip()
            chave_atual = chave
            lista_valores = []
            informacoes[chave] = valor
        elif linha.startswith("-"):
            valor = linha.replace("-", "").strip()
            lista_valores.append(valor)
    
    if chave_atual:
        informacoes[chave_atual] = lista_valores if lista_valores else valor.strip()

    return informacoes

In [4]:
caminho_arquivo = "profile.pdf"

if not os.path.exists(caminho_arquivo):
    raise FileNotFoundError(f"O arquivo {caminho_arquivo} não foi encontrado.")

In [5]:
documentos = carregar_documentos(caminho_arquivo)

informacoes_extraidas = []

for doc in documentos:
    texto_limpo = limpar_texto(doc.page_content)
    informacao = extrair_informacoes(texto_limpo)
    informacoes_extraidas.append(informacao)

informacoes_json = json.dumps(informacoes_extraidas, indent=4, ensure_ascii=False)
print(informacoes_json)


[
    {
        "Guia-prático-de-TypeScript": [
            "AWS Community Builders  Autor Técnico  CPA 20  Lider de"
        ],
        "In-my-publication,-\"Guia-prático-de-TypeScript": "Melhore suas"
    },
    {
        "As-a-Techlead-Developer,-I've-been-working-with": [
            "With .NET/C# API, .NET Core 1,2,3, .NET 5 and .NET 6 , Node.js with",
            "Creating pipes at Azure to work with Continuous integration and continuous",
            "Working with SQL Server using dapper and EF Core and NOSql with",
            "Manager three teams (System team, REC and BandPlay)"
        ]
    },
    {
        "Projects-that-we-migrate-last-year-to-new-environment": "",
        "*-https": "//smithsonianchannel.com.br/home (TypeScript + Node.js at Azure with",
        "As-a-Developer,-I've-been-working-with-some-really-cool-stuff-here": [
            "Developing with .NET/C# API, .NET Core, Node.js, Angular and others",
            "Continuous integration /  continuous delivery"

Analisando o perfil

In [6]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = Ollama(
    model="llama2",
    num_gpu=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

prompt_template = PromptTemplate(
    input_variables=["json_data"],
    template="Analise o seguinte currículo de um candidato e forneça uma avaliação resumida em porguês: {json_data}"
)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

resultado_analise = chain.invoke(informacoes_json)

/var/folders/8_/k1fm88l93q978ncwnjpl0rzr0000gn/T/ipykernel_17295/3094371105.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(


Avaliação resumida do currículo:

O candidato tem experiência em várias tecnologias, incluindo TypeScript, Node.js, .NET, C#, Angular e outros. Ele também tem experiência em continuidade de integração e continuidade de entrega.

A seção "Guia Prático de TypeScript" revela que o candidato é um autor técnico e líder de equipe com experiência em AWS Community Builders. Isso indica que ele tem habilidades como um profissional altamente capacitado e capaz de liderar equipes.

A seção "As um Técnico de Desenvolvimento" destaca a experiência do candidato em desenvolvimento com tecnologias como .NET/C#, Node.js, Angular e outros. Ele também tem experiência em integração de terceiros APIs, como Google e Facebook.

A seção "Projetos que migramos recentemente para novo ambiente" é vazia, o que pode indicar que ele não tem muito conhecimento sobre essa área.

A seção "As um Desenvolvedor, eu tenho trabalhado com alguma coisa incrível aqui" lista várias habilidades e tecnologias de desenvolvimento,

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import re

chat = ChatOpenAI(model='gpt-4-turbo')

prompt_template = PromptTemplate(
    input_variables=["analise"],
    template="""
    Você é um analista do RH (Recursos Humanos).
    Aqui está uma análise de currículo fornecida:
    {analise}
    
    De acordo com a análise do currículo, o candidato tem perfil para a posição de Tech Lead? Justifique sua resposta.
    """
)

def analise_rh(resultado_analise):
    resultado_analise_str = str(resultado_analise)    
    prompt = prompt_template.format(analise=resultado_analise_str)    
    resposta = chat.invoke(prompt)    
    return resposta


resposta = analise_rh(resultado_analise)

print(f"Resposta: {resposta}")


Resposta: content='Sim, o candidato parece ter um perfil adequado para a posição de Tech Lead, com base na análise do currículo fornecida. Aqui estão os principais pontos que justificam essa conclusão:\n\n1. **Experiência Técnica Abrangente**: O currículo destaca a experiência do candidato com uma variedade de tecnologias-chave, incluindo .NET (nas suas várias versões como .NET Core e .NET 5, 6), C#, TypeScript, Node.js e Angular. Essas são habilidades valiosas para um Tech Lead, que precisa ter um bom entendimento de várias tecnologias para orientar decisões técnicas e solucionar problemas.\n\n2. **Liderança de Equipe**: O candidato demonstrou habilidades de liderança ao gerenciar três equipes diferentes (System team, REC e BandPlay). A capacidade de liderar e gerenciar equipes é crucial para um Tech Lead, que deve coordenar esforços, motivar a equipe e assegurar que os projetos estejam alinhados com os objetivos organizacionais.\n\n3. **Experiência com Integração Contínua e Entrega C